In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd

options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)

driver.get("https://mytcas.com/")
time.sleep(3)

search_box = driver.find_element(By.XPATH, '//*[@id="search"]')
search_box.send_keys("วิศวกรรมคอมพิวเตอร์")
search_box.send_keys(Keys.ENTER) 

time.sleep(5)

names = []

for i in range(1, 58): 
    try:
        xpath = f'/html/body/main/div[1]/div[4]/div/ul/li[{i}]/a'
        element = driver.find_element(By.XPATH, xpath)
        text = element.text.strip()
        if text:
            names.append(text)
    except Exception as e:
        print(f"not found element i = {i}")

driver.quit()

df = pd.DataFrame(names, columns=["ชื่อหลักสูตร"])
df.to_excel("coe_courses.xlsx", index=False)

print("successfully")


successfully


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
import re

options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)

driver.get("https://mytcas.com/")
time.sleep(3)

search_box = driver.find_element(By.XPATH, '//*[@id="search"]')
search_box.send_keys("วิศวกรรมคอมพิวเตอร์")
search_box.send_keys(Keys.ENTER)
time.sleep(5)

courses = []

links = []
for i in range(1, 58):
    try:
        xpath = f'/html/body/main/div[1]/div[4]/div/ul/li[{i}]/a'
        element = driver.find_element(By.XPATH, xpath)
        title = element.text.strip()
        href = element.get_attribute("href")
        if title and href:
            links.append((title, href))
    except:
        print(f"element i={i}")

for title, href in links:
    driver.get(href)
    time.sleep(3)  

    try:
        fee_xpath = '//dt[contains(text(), "ค่าใช้จ่าย")]/following-sibling::dd[1]'
        fee_element = driver.find_element(By.XPATH, fee_xpath)
        fee_text = fee_element.text.strip()

        match = re.search(r'[\d,]+', fee_text)
        if match:
            fee = int(match.group().replace(',', ''))

            if fee > 280000:
                fee = int(fee / 8)

            if fee < 1000:
                fee = "ไม่พบข้อมูล"
        else:
            fee = "ไม่พบข้อมูล"

    except:
        fee = "ไม่พบข้อมูล"


    parts = title.split('\n')
    course_name = parts[0].strip() if len(parts) > 0 else ""
    faculty = parts[1].strip() if len(parts) > 1 else ""
    university = parts[2].strip() if len(parts) > 2 else ""

    courses.append({
        "ชื่อหลักสูตร": course_name,
        "คณะ": faculty,
        "มหาวิทยาลัย": university,
        "ค่าใช้จ่าย": fee
    })


driver.quit()

df = pd.DataFrame(courses)
df.to_excel("fee2.xlsx", index=False)

print("successfully")


successfully


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
import re

options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)

courses = []

def collect_courses(keyword, max_items):
    driver.get("https://mytcas.com/")
    time.sleep(3)

    search_box = driver.find_element(By.XPATH, '//*[@id="search"]')
    search_box.clear()
    search_box.send_keys(keyword)
    search_box.send_keys(Keys.ENTER)
    time.sleep(5)

    links = []
    for i in range(1, max_items):
        try:
            xpath = f'/html/body/main/div[1]/div[4]/div/ul/li[{i}]/a'
            element = driver.find_element(By.XPATH, xpath)
            title = element.text.strip()
            href = element.get_attribute("href")
            if title and href:
                links.append((title, href))
        except:
            print(f"ไม่พบ element i={i} ในคำค้น: {keyword}")

    for title, href in links:
        driver.get(href)
        time.sleep(3)  

        try:
            fee_xpath = '//dt[contains(text(), "ค่าใช้จ่าย")]/following-sibling::dd[1]'
            fee_element = driver.find_element(By.XPATH, fee_xpath)
            fee_text = fee_element.text.strip()

            match = re.search(r'[\d,]+', fee_text)
            if match:
                fee = int(match.group().replace(',', ''))

                if fee > 232700:
                    fee = int(fee / 8)

                if fee < 1000:
                    fee = "ไม่พบข้อมูล"
            else:
                fee = "ไม่พบข้อมูล"
        except:
            fee = "ไม่พบข้อมูล"

        # แยกข้อมูล title เป็น 3 ส่วน
        parts = title.split('\n')
        course_name = parts[0].strip() if len(parts) > 0 else ""
        faculty = parts[1].strip() if len(parts) > 1 else ""
        university = parts[2].strip() if len(parts) > 2 else ""

        courses.append({
            "ชื่อหลักสูตร": course_name,
            "คณะ": faculty,
            "มหาวิทยาลัย": university,
            "ค่าใช้จ่าย": fee,
        })

collect_courses("วิศวกรรมคอมพิวเตอร์", 58)

collect_courses("วิศวกรรมปัญญาประดิษฐ์", 8)

driver.quit()

df = pd.DataFrame(courses)
df.to_excel("coe_and_aie.xlsx", index=False)

print("successfully")


successfully


In [47]:
data = pd.DataFrame(courses)
data.head()

,ชื่อหลักสูตร,คณะ,มหาวิทยาลัย,ค่าใช้จ่าย
0,หลักสูตรวิศวกรรมศาสตรบัณฑิต สาขาวิชาวิศวกรรมคอ...,คณะวิศวกรรมศาสตร์ › วิศวกรรมคอมพิวเตอร์,จุฬาลงกรณ์มหาวิทยาลัย,25500
1,หลักสูตรวิศวกรรมศาสตรบัณฑิต สาขาวิชาวิศวกรรมคอ...,คณะวิศวกรรมศาสตร์ › วิศวกรรมคอมพิวเตอร์,จุฬาลงกรณ์มหาวิทยาลัย,25500
2,วศ.บ. สาขาวิชาวิศวกรรมคอมพิวเตอร์ (ภาษาไทย ปกติ),คณะวิศวกรรมศาสตร์ › วิศวกรรมคอมพิวเตอร์,มหาวิทยาลัยเกษตรศาสตร์ บางเขน,ไม่พบข้อมูล
3,วศ.บ. สาขาวิชาวิศวกรรมคอมพิวเตอร์ (ภาษาไทย ปกติ),คณะวิศวกรรมศาสตร์ กำแพงแสน › วิศวกรรมคอมพิวเตอร์,มหาวิทยาลัยเกษตรศาสตร์ กำแพงแสน,156000
4,วศ.บ. สาขาวิชาวิศวกรรมคอมพิวเตอร์และสารสนเทศศา...,คณะวิศวกรรมศาสตร์ ศรีราชา › วิศวกรรมคอมพิวเตอร์,มหาวิทยาลัยเกษตรศาสตร์ ศรีราชา,232800


In [48]:
data[["ค่าใช้จ่าย"]]

,ค่าใช้จ่าย
0,25500
1,25500
2,ไม่พบข้อมูล
3,156000
4,232800
...,...
59,ไม่พบข้อมูล
60,175000
61,28000
62,110000


In [65]:
filtered_data = data[data["ค่าใช้จ่าย"].apply(lambda x: isinstance(x, (int, float)))]
# print(filtered_data[["ค่าใช้จ่าย"]])
# filtered_data.describe()
# filtered_data[["ค่าใช้จ่าย"]].min()
filtered_data[["ค่าใช้จ่าย"]].max()

ค่าใช้จ่าย    232800
dtype: object

In [75]:
filtered_data = data[
    data["ค่าใช้จ่าย"].apply(lambda x: isinstance(x, (int, float)) and x > 100000)
]


filtered_data[["ค่าใช้จ่าย"]]

,ค่าใช้จ่าย
3,156000
4,232800
7,184000
10,101200
12,200000
25,197100
29,112000
45,110000
46,110000
47,110000


In [78]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
import re

options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)

courses = []

def collect_courses(keyword, max_items):
    driver.get("https://mytcas.com/")
    time.sleep(3)

    search_box = driver.find_element(By.XPATH, '//*[@id="search"]')
    search_box.clear()
    search_box.send_keys(keyword)
    search_box.send_keys(Keys.ENTER)
    time.sleep(5)

    links = []
    for i in range(1, max_items):
        try:
            xpath = f'/html/body/main/div[1]/div[4]/div/ul/li[{i}]/a'
            element = driver.find_element(By.XPATH, xpath)
            title = element.text.strip()
            href = element.get_attribute("href")
            if title and href:
                links.append((title, href))
        except:
            print(f"ไม่พบ element i={i} ในคำค้น: {keyword}")

    for title, href in links:
        driver.get(href)
        time.sleep(3)  

        # แยกข้อมูล title เป็น 3 ส่วน (ก่อนคำนวณ fee เพื่อใช้เงื่อนไขภาษาไทย)
        parts = title.split('\n')
        course_name = parts[0].strip() if len(parts) > 0 else ""
        faculty = parts[1].strip() if len(parts) > 1 else ""
        university = parts[2].strip() if len(parts) > 2 else ""

        try:
            fee_xpath = '//dt[contains(text(), "ค่าใช้จ่าย")]/following-sibling::dd[1]'
            fee_element = driver.find_element(By.XPATH, fee_xpath)
            fee_text = fee_element.text.strip()

            match = re.search(r'[\d,]+', fee_text)
            if match:
                fee = int(match.group().replace(',', ''))

                if "ภาษาไทย" in course_name and fee > 50000:
                    fee = int(fee / 8)
                    
                if fee > 280000:
                    fee = int(fee / 8)

                if fee < 1000:
                    fee = "ไม่พบข้อมูล"
            else:
                fee = "ไม่พบข้อมูล"

        except:
            fee = "ไม่พบข้อมูล"

        courses.append({
            "ชื่อหลักสูตร": course_name,
            "คณะ": faculty,
            "มหาวิทยาลัย": university,
            "ค่าใช้จ่าย": fee
        })

collect_courses("วิศวกรรมคอมพิวเตอร์", 58)

collect_courses("วิศวกรรมปัญญาประดิษฐ์", 8)

driver.quit()

df = pd.DataFrame(courses)
df.to_excel("coe_and_aie_new.xlsx", index=False)

print("successfully")


successfully


In [16]:
import pandas as pd
df = pd.read_excel("coe_and_aie_new.xlsx")
df

,ชื่อหลักสูตร,คณะ,มหาวิทยาลัย,ค่าใช้จ่าย
0,หลักสูตรวิศวกรรมศาสตรบัณฑิต สาขาวิชาวิศวกรรมคอ...,คณะวิศวกรรมศาสตร์ › วิศวกรรมคอมพิวเตอร์,จุฬาลงกรณ์มหาวิทยาลัย,25500
1,หลักสูตรวิศวกรรมศาสตรบัณฑิต สาขาวิชาวิศวกรรมคอ...,คณะวิศวกรรมศาสตร์ › วิศวกรรมคอมพิวเตอร์,จุฬาลงกรณ์มหาวิทยาลัย,25500
2,วศ.บ. สาขาวิชาวิศวกรรมคอมพิวเตอร์ (ภาษาไทย ปกติ),คณะวิศวกรรมศาสตร์ › วิศวกรรมคอมพิวเตอร์,มหาวิทยาลัยเกษตรศาสตร์ บางเขน,ไม่พบข้อมูล
3,วศ.บ. สาขาวิชาวิศวกรรมคอมพิวเตอร์ (ภาษาไทย ปกติ),คณะวิศวกรรมศาสตร์ กำแพงแสน › วิศวกรรมคอมพิวเตอร์,มหาวิทยาลัยเกษตรศาสตร์ กำแพงแสน,19500
4,วศ.บ. สาขาวิชาวิศวกรรมคอมพิวเตอร์และสารสนเทศศา...,คณะวิศวกรรมศาสตร์ ศรีราชา › วิศวกรรมคอมพิวเตอร์,มหาวิทยาลัยเกษตรศาสตร์ ศรีราชา,29100
...,...,...,...,...
59,หลักสูตรวิศวกรรมศาสตรบัณฑิต สาขาวิชาวิศวกรรมระ...,วิทยาลัยสหวิทยาการ › วิศวกรรมปัญญาประดิษฐ์,มหาวิทยาลัยเทคโนโลยีพระจอมเกล้าธนบุรี บางมด,ไม่พบข้อมูล
60,วศ.บ. วิศวกรรมปัญญาประดิษฐ์และการเป็นผู้ประกอบ...,คณะวิศวกรรมศาสตร์ › วิศวกรรมปัญญาประดิษฐ์,สถาบันเทคโนโลยีพระจอมเกล้าเจ้าคุณทหารลาดกระบัง...,175000
61,วศ.บ.สาขาวิชาวิศวกรรมปัญญาประดิษฐ์และการสั่งกา...,คณะวิศวกรรมศาสตร์ › วิศวกรรมปัญญาประดิษฐ์,มหาวิทยาลัยอุบลราชธานี วิทยาเขตหลัก,28000
62,วศ.บ. วิศวกรรมปัญญาประดิษฐ์และนวัตกรรมดิจิทัล ...,คณะวิศวกรรมศาสตร์ › วิศวกรรมคอมพิวเตอร์,มหาวิทยาลัยเทคโนโลยีราชมงคลศรีวิชัย สงขลา,13750


In [17]:
pd.to_numeric(df['ค่าใช้จ่าย'], errors='coerce').min()



9025.0

In [18]:
# แปลงเป็นตัวเลข (string ที่แปลงไม่ได้จะเป็น NaN)
df["ค่าใช้จ่าย"] = pd.to_numeric(df["ค่าใช้จ่าย"], errors='coerce')

# ลบแถวที่เป็น NaN (เช่น "ไม่พบข้อมูล")
df = df.dropna(subset=["ค่าใช้จ่าย"])
df["ค่าใช้จ่าย"]


0      25500.0
1      25500.0
3      19500.0
4      29100.0
6      20000.0
7      23000.0
8      50000.0
9      18900.0
10    101200.0
11     30000.0
12     25000.0
13     18000.0
14     28000.0
15     48000.0
19     25000.0
20     25000.0
21     90000.0
22     50000.0
23     20000.0
25     24637.0
26     16000.0
27     32700.0
28     28000.0
29     14000.0
30      9025.0
32     10150.0
33     33000.0
34     50407.0
35     31500.0
36     12000.0
37     18500.0
38      9076.0
39     12500.0
40     20000.0
41     16120.0
42     15000.0
43     15000.0
44     15000.0
45     13750.0
46     13750.0
47     13750.0
48     12000.0
50     42500.0
52     49600.0
53     42125.0
54     35250.0
55     37937.0
56     48000.0
57     28000.0
58     38000.0
60    175000.0
61     28000.0
62     13750.0
63     48000.0
Name: ค่าใช้จ่าย, dtype: float64

In [19]:
duplicates = df[df.duplicated(subset=["ชื่อหลักสูตร", "คณะ", "มหาวิทยาลัย", "ค่าใช้จ่าย"], keep=False)]
print(duplicates)


                                         ชื่อหลักสูตร  \
14         วศ.บ. วิศวกรรมปัญญาประดิษฐ์ (ภาษาไทย ปกติ)   
46  วศ.บ. วิศวกรรมปัญญาประดิษฐ์และนวัตกรรมดิจิทัล ...   
56  หลักสูตรวิศวกรรมศาสตรบัณฑิต สาขาวิชาวิศวกรรมคอ...   
57         วศ.บ. วิศวกรรมปัญญาประดิษฐ์ (ภาษาไทย ปกติ)   
62  วศ.บ. วิศวกรรมปัญญาประดิษฐ์และนวัตกรรมดิจิทัล ...   
63  หลักสูตรวิศวกรรมศาสตรบัณฑิต สาขาวิชาวิศวกรรมคอ...   

                                                  คณะ  \
14            คณะวิศวกรรมศาสตร์ › วิศวกรรมคอมพิวเตอร์   
46            คณะวิศวกรรมศาสตร์ › วิศวกรรมคอมพิวเตอร์   
56  คณะวิศวกรรมศาสตร์และเทคโนโลยี › วิศวกรรมปัญญาป...   
57            คณะวิศวกรรมศาสตร์ › วิศวกรรมคอมพิวเตอร์   
62            คณะวิศวกรรมศาสตร์ › วิศวกรรมคอมพิวเตอร์   
63  คณะวิศวกรรมศาสตร์และเทคโนโลยี › วิศวกรรมปัญญาป...   

                                      มหาวิทยาลัย  ค่าใช้จ่าย  
14               มหาวิทยาลัยสงขลานครินทร์ หาดใหญ่     28000.0  
46      มหาวิทยาลัยเทคโนโลยีราชมงคลศรีวิชัย สงขลา     13750.0  
56  สถาบ

In [20]:
df = df.drop_duplicates(subset=["ชื่อหลักสูตร", "คณะ", "มหาวิทยาลัย", "ค่าใช้จ่าย"], keep='first')
df

,ชื่อหลักสูตร,คณะ,มหาวิทยาลัย,ค่าใช้จ่าย
0,หลักสูตรวิศวกรรมศาสตรบัณฑิต สาขาวิชาวิศวกรรมคอ...,คณะวิศวกรรมศาสตร์ › วิศวกรรมคอมพิวเตอร์,จุฬาลงกรณ์มหาวิทยาลัย,25500.0
1,หลักสูตรวิศวกรรมศาสตรบัณฑิต สาขาวิชาวิศวกรรมคอ...,คณะวิศวกรรมศาสตร์ › วิศวกรรมคอมพิวเตอร์,จุฬาลงกรณ์มหาวิทยาลัย,25500.0
3,วศ.บ. สาขาวิชาวิศวกรรมคอมพิวเตอร์ (ภาษาไทย ปกติ),คณะวิศวกรรมศาสตร์ กำแพงแสน › วิศวกรรมคอมพิวเตอร์,มหาวิทยาลัยเกษตรศาสตร์ กำแพงแสน,19500.0
4,วศ.บ. สาขาวิชาวิศวกรรมคอมพิวเตอร์และสารสนเทศศา...,คณะวิศวกรรมศาสตร์ ศรีราชา › วิศวกรรมคอมพิวเตอร์,มหาวิทยาลัยเกษตรศาสตร์ ศรีราชา,29100.0
6,วิศวกรรมศาสตรบัณฑิต (วิศวกรรมคอมพิวเตอร์) วิศว...,คณะวิศวกรรมศาสตร์ › วิศวกรรมคอมพิวเตอร์,มหาวิทยาลัยขอนแก่น ขอนแก่น,20000.0
7,วศ.บ.วิศวกรรมคอมพิวเตอร์ (ภาษาไทย ปกติ),คณะวิศวกรรมศาสตร์ › วิศวกรรมคอมพิวเตอร์,มหาวิทยาลัยเชียงใหม่ วิทยาเขตหลัก,23000.0
8,วศ.บ.วิศวกรรมระบบสารสนเทศและเครือข่าย (นานาชาติ),คณะวิศวกรรมศาสตร์ › วิศวกรรมคอมพิวเตอร์,มหาวิทยาลัยเชียงใหม่ วิทยาเขตหลัก,50000.0
9,วศ.บ.สาขาวิชาวิศวกรรมคอมพิวเตอร์ (ภาษาไทย ปกติ),คณะวิศวกรรมศาสตร์ › วิศวกรรมคอมพิวเตอร์,มหาวิทยาลัยธรรมศาสตร์ ศูนย์รังสิต,18900.0
10,วศ.บ.สาขาวิชาวิศวกรรมดิจิทัล และวิศวกรรมคอมพิว...,สถาบันเทคโนโลยีนานาชาติสิรินธร › วิศวกรรมคอมพิ...,มหาวิทยาลัยธรรมศาสตร์ ศูนย์รังสิต,101200.0
11,หลักสูตรวิศวกรรมศาสตรบัณฑิต สาขาวิชาวิศวกรรมคอ...,คณะวิศวกรรมศาสตร์ › วิศวกรรมคอมพิวเตอร์,มหาวิทยาลัยมหิดล ศาลายา,30000.0


In [28]:
df = pd.read_excel("coe_and_aie_clean.xlsx")
df

,ชื่อหลักสูตร,คณะ,มหาวิทยาลัย,ค่าใช้จ่าย
0,หลักสูตรวิศวกรรมศาสตรบัณฑิต สาขาวิชาวิศวกรรมคอ...,คณะวิศวกรรมศาสตร์ › วิศวกรรมคอมพิวเตอร์,จุฬาลงกรณ์มหาวิทยาลัย,25500
1,หลักสูตรวิศวกรรมศาสตรบัณฑิต สาขาวิชาวิศวกรรมคอ...,คณะวิศวกรรมศาสตร์ › วิศวกรรมคอมพิวเตอร์,จุฬาลงกรณ์มหาวิทยาลัย,25500
2,วศ.บ. สาขาวิชาวิศวกรรมคอมพิวเตอร์ (ภาษาไทย ปกติ),คณะวิศวกรรมศาสตร์ กำแพงแสน › วิศวกรรมคอมพิวเตอร์,มหาวิทยาลัยเกษตรศาสตร์ กำแพงแสน,19500
3,วศ.บ. สาขาวิชาวิศวกรรมคอมพิวเตอร์และสารสนเทศศา...,คณะวิศวกรรมศาสตร์ ศรีราชา › วิศวกรรมคอมพิวเตอร์,มหาวิทยาลัยเกษตรศาสตร์ ศรีราชา,29100
4,วิศวกรรมศาสตรบัณฑิต (วิศวกรรมคอมพิวเตอร์) วิศว...,คณะวิศวกรรมศาสตร์ › วิศวกรรมคอมพิวเตอร์,มหาวิทยาลัยขอนแก่น ขอนแก่น,20000
5,วศ.บ.วิศวกรรมคอมพิวเตอร์ (ภาษาไทย ปกติ),คณะวิศวกรรมศาสตร์ › วิศวกรรมคอมพิวเตอร์,มหาวิทยาลัยเชียงใหม่ วิทยาเขตหลัก,23000
6,วศ.บ.วิศวกรรมระบบสารสนเทศและเครือข่าย (นานาชาติ),คณะวิศวกรรมศาสตร์ › วิศวกรรมคอมพิวเตอร์,มหาวิทยาลัยเชียงใหม่ วิทยาเขตหลัก,50000
7,วศ.บ.สาขาวิชาวิศวกรรมคอมพิวเตอร์ (ภาษาไทย ปกติ),คณะวิศวกรรมศาสตร์ › วิศวกรรมคอมพิวเตอร์,มหาวิทยาลัยธรรมศาสตร์ ศูนย์รังสิต,18900
8,วศ.บ.สาขาวิชาวิศวกรรมดิจิทัล และวิศวกรรมคอมพิว...,สถาบันเทคโนโลยีนานาชาติสิรินธร › วิศวกรรมคอมพิ...,มหาวิทยาลัยธรรมศาสตร์ ศูนย์รังสิต,101200
9,หลักสูตรวิศวกรรมศาสตรบัณฑิต สาขาวิชาวิศวกรรมคอ...,คณะวิศวกรรมศาสตร์ › วิศวกรรมคอมพิวเตอร์,มหาวิทยาลัยมหิดล ศาลายา,30000


In [ ]:
# กรอclงแถวที่ชื่อหลักสูตรมีคำว่า "นานาติ" (ไม่สนใจตัวพิมพ์เล็ก-ใหญ่)
filtered_df = df[df["มหาวิทยาลัย"].str.contains("กรุงเทพ", case=False, na=False)]

filtered_df[["ชื่อหลักสูตร", "ค่าใช้จ่าย", "คณะ", "มหาวิทยาลัย"]]


,ชื่อหลักสูตร,ค่าใช้จ่าย,คณะ,มหาวิทยาลัย
14,วศ.บ.วิศวกรรมคอมพิวเตอร์ (ภาษาไทย ปกติ),25000,คณะวิศวกรรมศาสตร์ › วิศวกรรมคอมพิวเตอร์,มหาวิทยาลัยเทคโนโลยีพระจอมเกล้าพระนครเหนือ กรุ...
34,วศ.บ. (วิศวกรรมคอมพิวเตอร์และระบบไอโอที) (ภาษา...,16120,คณะวิศวกรรมศาสตร์ › วิศวกรรมคอมพิวเตอร์,มหาวิทยาลัยเทคโนโลยีราชมงคลกรุงเทพ


In [1]:
import pandas as pd

# โหลดไฟล์ Excel
df = pd.read_excel("coe_and_aie_clean.xlsx")

# สร้าง mapping ของมหาวิทยาลัย → ภาค
university_region_map = {
    'จุฬาลงกรณ์มหาวิทยาลัย': 'ภาคกลาง',
    'มหาวิทยาลัยเกษตรศาสตร์ กำแพงแสน': 'ภาคกลาง',
    'มหาวิทยาลัยเกษตรศาสตร์ ศรีราชา': 'ภาคตะวันออก',
    'มหาวิทยาลัยขอนแก่น ขอนแก่น': 'ภาคตะวันออกเฉียงเหนือ',
    'มหาวิทยาลัยเชียงใหม่': 'ภาคเหนือ',
    'มหาวิทยาลัยสงขลานครินทร์': 'ภาคใต้',
    'มหาวิทยาลัยเทคโนโลยีพระจอมเกล้าธนบุรี': 'ภาคกลาง',
    'มหาวิทยาลัยเทคโนโลยีพระจอมเกล้าพระนครเหนือ': 'ภาคกลาง',
    'มหาวิทยาลัยเทคโนโลยีสุรนารี': 'ภาคตะวันออกเฉียงเหนือ',
    'มหาวิทยาลัยบูรพา': 'ภาคตะวันออก',
    'มหาวิทยาลัยมหาสารคาม': 'ภาคตะวันออกเฉียงเหนือ',
    'มหาวิทยาลัยศรีนครินทรวิโรฒ': 'ภาคกลาง',
    'มหาวิทยาลัยนเรศวร': 'ภาคเหนือ',
    'มหาวิทยาลัยธรรมศาสตร์ ศูนย์รังสิต': 'ภาคกลาง',
    'มหาวิทยาลัยแม่ฟ้าหลวง': 'ภาคเหนือ',
    'มหาวิทยาลัยวลัยลักษณ์': 'ภาคใต้',
    'มหาวิทยาลัยนวมินทราธิราช': 'ภาคกลาง',
    'มหาวิทยาลัยเทคโนโลยีมหานคร': 'ภาคกลาง',
    'มหาวิทยาลัยหาดใหญ่': 'ภาคใต้',
    'มหาวิทยาลัยพะเยา': 'ภาคเหนือ',
    'มหาวิทยาลัยรังสิต': 'ภาคกลาง',
    'มหาวิทยาลัยศิลปากร': 'ภาคกลาง',
    'มหาวิทยาลัยสยาม': 'ภาคกลาง',
    'มหาวิทยาลัยเอเชียอาคเนย์': 'ภาคกลาง',
    'มหาวิทยาลัยเทคโนโลยีราชมงคลธัญบุรี': 'ภาคกลาง',
    'มหาวิทยาลัยราชภัฏนครราชสีมา': 'ภาคตะวันออกเฉียงเหนือ',
    'มหาวิทยาลัยราชภัฏอุบลราชธานี': 'ภาคตะวันออกเฉียงเหนือ',
    'มหาวิทยาลัยราชภัฏเชียงใหม่': 'ภาคเหนือ',

    # เพิ่มจากการตรวจสอบล่าสุด
    'มหาวิทยาลัยเชียงใหม่ วิทยาเขตหลัก': 'ภาคเหนือ',
    'มหาวิทยาลัยมหิดล ศาลายา': 'ภาคกลาง',
    'มหาวิทยาลัยศรีนครินทรวิโรฒ องครักษ์': 'ภาคกลาง',
    'มหาวิทยาลัยสงขลานครินทร์ หาดใหญ่': 'ภาคใต้',
    'มหาวิทยาลัยสงขลานครินทร์ ภูเก็ต': 'ภาคใต้',
    'มหาวิทยาลัยเทคโนโลยีพระจอมเกล้าพระนครเหนือ กรุงเทพฯ': 'ภาคกลาง',
    'สถาบันเทคโนโลยีพระจอมเกล้าเจ้าคุณทหารลาดกระบัง ลาดกระบัง': 'ภาคกลาง',
    'สถาบันเทคโนโลยีพระจอมเกล้าเจ้าคุณทหารลาดกระบัง ชุมพรเขตรอุดมศักดิ์ จังหวัดชุมพร': 'ภาคใต้',
    'มหาวิทยาลัยเทคโนโลยีสุรนารี วิทยาเขตหลัก': 'ภาคตะวันออกเฉียงเหนือ',
    'มหาวิทยาลัยนเรศวร วิทยาเขตหลัก': 'ภาคเหนือ',
    'มหาวิทยาลัยนราธิวาสราชนครินทร์': 'ภาคใต้',
    'มหาวิทยาลัยนครพนม': 'ภาคตะวันออกเฉียงเหนือ',
    'มหาวิทยาลัยกาฬสินธุ์ วิทยาเขตหลัก': 'ภาคตะวันออกเฉียงเหนือ',
    'มหาวิทยาลัยรามคำแหง': 'ภาคกลาง',
    'สถาบันเทคโนโลยีจิตรลดา': 'ภาคกลาง',
    'มหาวิทยาลัยราชภัฏพิบูลสงคราม': 'ภาคเหนือ',
    'มหาวิทยาลัยราชภัฏสวนสุนันทา': 'ภาคกลาง',
    'มหาวิทยาลัยราชภัฏอุบลราชธานี วิทยาเขตหลัก': 'ภาคตะวันออกเฉียงเหนือ',
    'มหาวิทยาลัยเทคโนโลยีราชมงคลกรุงเทพ': 'ภาคกลาง',
    'มหาวิทยาลัยเทคโนโลยีราชมงคลพระนคร พระนครเหนือ': 'ภาคกลาง',
    'มหาวิทยาลัยเทคโนโลยีราชมงคลรัตนโกสินทร์ ศาลายา': 'ภาคกลาง',
    'มหาวิทยาลัยเทคโนโลยีราชมงคลรัตนโกสินทร์ วังไกลกังวล': 'ภาคตะวันตก',
    'มหาวิทยาลัยเทคโนโลยีราชมงคลศรีวิชัย สงขลา': 'ภาคใต้',
    'มหาวิทยาลัยเทคโนโลยีราชมงคลศรีวิชัย ตรัง': 'ภาคใต้',
    'มหาวิทยาลัยเทคโนโลยีราชมงคลอีสาน นครราชสีมา': 'ภาคตะวันออกเฉียงเหนือ',
    'มหาวิทยาลัยหอการค้าไทย': 'ภาคกลาง',
    'มหาวิทยาลัยธุรกิจบัณฑิตย์': 'ภาคกลาง',
    'มหาวิทยาลัยเกษมบัณฑิต วิทยาเขตร่มเกล้า': 'ภาคกลาง',
    'มหาวิทยาลัยสยาม วิทยาเขตหลัก': 'ภาคกลาง',
    'สถาบันการจัดการปัญญาภิวัฒน์ แจ้งวัฒนะ นนทบุรี': 'ภาคกลาง',
    'มหาวิทยาลัยอุบลราชธานี วิทยาเขตหลัก': 'ภาคตะวันออกเฉียงเหนือ',
}

# เพิ่มคอลัมน์ภาค
df['ภาค'] = df['มหาวิทยาลัย'].map(university_region_map)

# เพิ่มคอลัมน์ภาษา
df['หลักสูตร'] = df['ชื่อหลักสูตร'].apply(lambda x: 'นานาชาติ' if 'นานาชาติ' in str(x) else 'ภาษาไทย')

# บันทึกไฟล์ใหม่
df.to_excel("coe_and_aie_with_region_language.xlsx", index=False)
print("สร้างไฟล์เรียบร้อย: coe_and_aie_with_region_language.xlsx")


สร้างไฟล์เรียบร้อย: coe_and_aie_with_region_language.xlsx


In [1]:
import pandas as pd

# โหลดไฟล์ Excel
df = pd.read_excel("coe_and_aie_with_region_language.xlsx")

# สร้างคอลัมน์ใหม่ "ชื่อสาขา"
df["ชื่อสาขา"] = df["ชื่อหลักสูตร"].apply(
    lambda x: "วิศวกรรมปัญญาประดิษฐ์" if "ปัญญาประดิษฐ์" in str(x) else "วิศวกรรมคอมพิวเตอร์"
)

# ตรวจสอบผลลัพธ์
print(df[["ชื่อหลักสูตร", "ชื่อสาขา"]].head())

# (ถ้าต้องการบันทึกกลับลงไฟล์)
df.to_excel("coe_and_aie_with_major.xlsx", index=False)


                                        ชื่อหลักสูตร             ชื่อสาขา
0  หลักสูตรวิศวกรรมศาสตรบัณฑิต สาขาวิชาวิศวกรรมคอ...  วิศวกรรมคอมพิวเตอร์
1  หลักสูตรวิศวกรรมศาสตรบัณฑิต สาขาวิชาวิศวกรรมคอ...  วิศวกรรมคอมพิวเตอร์
2   วศ.บ. สาขาวิชาวิศวกรรมคอมพิวเตอร์ (ภาษาไทย ปกติ)  วิศวกรรมคอมพิวเตอร์
3  วศ.บ. สาขาวิชาวิศวกรรมคอมพิวเตอร์และสารสนเทศศา...  วิศวกรรมคอมพิวเตอร์
4  วิศวกรรมศาสตรบัณฑิต (วิศวกรรมคอมพิวเตอร์) วิศว...  วิศวกรรมคอมพิวเตอร์
